In [1]:
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.callbacks import CSVLogger

/Users/kentoyamamoto/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/kentoyamamoto/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/kentoyamamoto/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qu

In [36]:
n_categories = 5  # 10種類の野菜
batch_size = 32
train_dir = './images/train/'
validation_dir = './images/validation/'
file_name = 'vgg16_veg_file'

# 全結合層はいらないのでinclude_top=False
base_model = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224,224,3)))

# 全結合層を付け加える
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
prediction = Dense(n_categories, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=prediction)

# 14層目までを凍結
for layer in base_model.layers[:15]:
    layer.trainable = False

# 確率的勾配降下法
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# モデルをjsonで保存
json_string = model.to_json()
f = open('vgg16_veg_fine.json', 'w')
f.write(json_string)
f.close()

Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0  

In [37]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(
    rescale=1.0/ 255
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224,224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

Found 222 images belonging to 5 classes.
Found 99 images belonging to 5 classes.


In [39]:
hist = model.fit_generator(
    train_generator,
    epochs=50,
    verbose=1,
    validation_data=validation_generator,
    callbacks=[CSVLogger(file_name+'.csv')]
)
model.save(file_name+'.h5')

Epoch 1/50


/Users/kentoyamamoto/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/PIL/Image.py:918: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


7/7 [==============================] - 75s 11s/step - loss: 1.6529 - acc: 0.1930 - val_loss: 1.6191 - val_acc: 0.2020
Epoch 2/50
7/7 [==============================] - 73s 10s/step - loss: 1.6250 - acc: 0.2061 - val_loss: 1.5919 - val_acc: 0.2222
Epoch 3/50
7/7 [==============================] - 104s 15s/step - loss: 1.5947 - acc: 0.2424 - val_loss: 1.5650 - val_acc: 0.2727
Epoch 4/50
7/7 [==============================] - 83s 12s/step - loss: 1.5695 - acc: 0.2926 - val_loss: 1.5470 - val_acc: 0.3131
Epoch 5/50
7/7 [==============================] - 1531s 219s/step - loss: 1.5606 - acc: 0.2842 - val_loss: 1.5330 - val_acc: 0.3131
Epoch 6/50
7/7 [==============================] - 81s 12s/step - loss: 1.5424 - acc: 0.2879 - val_loss: 1.5213 - val_acc: 0.3131
Epoch 7/50
7/7 [==============================] - 84s 12s/step - loss: 1.5237 - acc: 0.2921 - val_loss: 1.5094 - val_acc: 0.3131
Epoch 8/50
7/7 [==============================] - 96s 14s/step - loss: 1.5154 - acc: 0.2918 - val_loss: 

In [2]:
from keras.models import model_from_json
import matplotlib.pyplot as plt
import numpy as np
import os, random
from keras.preprocessing.image import img_to_array, load_img
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD

batch_size = 32
file_name = "vgg16_veg_file"
test_dir = './images/test'
display_dir = './images/display'

# trainディレクトリのフォルダ名の順序通り並べます
label = ['carrot', 'meat', 'onion', 'potato', 'tomato']

json_string = open('vgg16_veg_fine.json').read()
model = model_from_json(json_string)
model.load_weights(file_name + '.h5')

model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
#test_datagen = ImageDataGenerator(rescale=1.0/255)

#test_generator = test_datagen.flow_from_directory(
#    test_dir,
#    target_size=(224,224),
#    batch_size=batch_size,
#    class_mode='categorical',
#    shuffle=True
#)

#score = model.evaluate_generator(test_generator)
#print('\n test loss:', score[0])
#print('\n test acc :', score[1])

files = os.listdir(display_dir)
img = random.sample(files, 25)

plt.figure(figsize=(10,10))
for i in range(25):
    temp_img = load_img(os.path.join(display_dir, img[i]), target_size=(224,224))
    plt.subplot(5,5,i+1)
    plt.imshow(temp_img)
    temp_img_array = img_to_array(temp_img)
    temp_img_array = temp_img_array.astype('float32') / 255.0
    temp_img_array = temp_img_array.reshape((1,224,224,3))

    img_pred = model.predict(temp_img_array)
    print(label[np.argmax(img_pred)])
    plt.title(label[np.argmax(img_pred)])
    plt.xticks([]), plt.yticks([])

plt.show()








tomato
tomato
tomato
tomato
tomato
meat
tomato
tomato
tomato
tomato
tomato
tomato
tomato
tomato
tomato
tomato
tomato
tomato
tomato
tomato
tomato
tomato
tomato
tomato
tomato


<Figure size 1000x1000 with 25 Axes>

In [1]:
from keras.models import model_from_json
import matplotlib.pyplot as plt
import numpy as np
import os, random
from keras.preprocessing.image import img_to_array, load_img
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD

batch_size = 32
file_name = "vgg16_veg_file"
test_dir = './images/test'
display_dir = './images/display'

# trainディレクトリのフォルダ名の順序通り並べます
label = ['carrot', 'meat', 'onion', 'potato', 'tomato']

json_string = open('./json/vgg16_veg_file.json').read()
model = model_from_json(json_string)
model.load_weights(file_name + '.h5')

#model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
#test_datagen = ImageDataGenerator(rescale=1.0/255)

#test_generator = test_datagen.flow_from_directory(
#    test_dir,
#    target_size=(224,224),
#    batch_size=batch_size,
#    class_mode='categorical',
#    shuffle=True
#)

#score = model.evaluate_generator(test_generator)
#print('\n test loss:', score[0])
#print('\n test acc :', score[1])

img = load_img("./images/display/img_2.jpg", grayscale=False,target_size=(224,224))


temp_img_array = img_to_array(img)
temp_img_array = temp_img_array.astype('float32') / 255.0
temp_img_array = temp_img_array.reshape((1,224,224,3))

img_pred = model.predict(temp_img_array)
print(label[np.argmax(img_pred)])


/Users/kentoyamamoto/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/kentoyamamoto/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/kentoyamamoto/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qu







tomato
